In [3]:
import numpy as np

# Degree distribution: pk[k] = P(degree = k)
pk = {1: 0.2, 2: 0.3, 3: 0.3, 4: 0.2}

# mean degree z = sum(k * pk)
z = sum(k * p for k, p in pk.items())

# Example of p_k^(n-1) = probability node of degree k active at time n-1
p_active = {1: 0.1, 2: 0.2, 3: 0.3, 4: 0.4}

# Compute p̄_{n−1}
p_bar_prev = sum((k / z) * pk[k] * p_active[k] for k in pk)
print("p̄_{n−1} =", p_bar_prev)

# Example C1^k (prob. that at least one neighbor is active)
C1 = {1: 0, 2: 0.3, 3: 0.4, 4: 0.6}

# θ_config
theta_config = sum((k / z) * pk[k] * (k - 1) * C1[k] for k in pk if k > 1)
print("θ_config =", theta_config)

# Example DB matrix (toy 4x4)
DB = np.array([
    [0.1, 0.2, 0.0, 0.0],
    [0.3, 0.1, 0.2, 0.0],
    [0.0, 0.4, 0.1, 0.3],
    [0.0, 0.0, 0.2, 0.1]
])

# λ_max = largest eigenvalue of DB
lambda_max = max(np.real(np.linalg.eigvals(DB)))
theta_crit = 1 / lambda_max
print("θ_crit (from eigenvalue) =", theta_crit)


p̄_{n−1} = 0.29200000000000004
θ_config = 0.936
θ_crit (from eigenvalue) = 1.9074356983054626


In [4]:
# hypergraph_critical_threshold.py
import xgi
import numpy as np
from scipy.sparse.linalg import eigs

def compute_DB_matrix(H):
    """
    Construct the DB matrix for a hypergraph H.
    Each hyperedge contributes weight 1/(|e|-1) between all its node pairs.
    """
    nodes = list(H.nodes)
    node_index = {node: idx for idx, node in enumerate(nodes)}
    n = len(nodes)
    DB = np.zeros((n, n))

    for e in H.edges.members():
        size = len(e)
        if size <= 1:
            continue
        weight = 1 / (size - 1)
        for i in e:
            for j in e:
                if i != j:
                    DB[node_index[i], node_index[j]] += weight
    return DB, nodes


def compute_theta_crit(DB):
    """Compute critical theta = 1 / λ_max(DB)"""
    eigvals = np.linalg.eigvals(DB)
    lambda_max = np.max(np.real(eigvals))
    theta_crit = 1 / lambda_max if lambda_max > 0 else np.inf
    return lambda_max, theta_crit


def summarize_hypergraph(H):
    """Print quick summary of structure"""
    degrees = [H.degree(node) for node in H.nodes]
    edge_sizes = [len(e) for e in H.edges.members()]
    z = np.mean(degrees)
    print(f"Mean hyperdegree (z): {z:.3f}")
    print(f"Average hyperedge size: {np.mean(edge_sizes):.3f}")
    print(f"Num nodes: {H.num_nodes}, Num edges: {H.num_edges}")


if __name__ == "__main__":
    # Load a sample hypergraph 
    H = xgi.load_xgi_data("hospital-lyon", max_order=None)

    summarize_hypergraph(H)
    DB, nodes = compute_DB_matrix(H)

    lambda_max, theta_crit = compute_theta_crit(DB)
    print(f"λ_max(DB): {lambda_max:.5f}")
    print(f"θ_crit ≈ {theta_crit:.5f}")


Mean hyperdegree (z): 774.680
Average hyperedge size: 2.087
Num nodes: 75, Num edges: 27834
λ_max(DB): 1823.05156
θ_crit ≈ 0.00055


In [11]:
datasets = [
    "contact-primary-school",
    "contact-high-school",
    "hospital-lyon",
    "email-enron",
    "email-eu",
    "ndc-substances",
    "diseasome",
    "congress-bills",
    "tags-ask-ubuntu",
]

#disgenenet is not included

In [6]:
# epidemic threshold for an SIR model
import xgi
import numpy as np
from scipy.sparse import lil_matrix
from scipy.sparse.linalg import eigs

def compute_DB_matrix(H):
    """
    Construct the DB matrix for a hypergraph H.
    Each hyperedge contributes weight 1/(|e|-1) between all its node pairs.
    """
    nodes = list(H.nodes)
    node_index = {node: idx for idx, node in enumerate(nodes)}
    n = len(nodes)

    # Initialize DB as sparse matrix
    DB = lil_matrix((n, n))

    # Loop through all hyperedges
    for e in H.edges.members():
        size = len(e)
        if size <= 1:
            continue # Skip single-node hyperedges

        # Weight for each node pair in this hyperedge
        weight = 1 / (size - 1)
        # Add weight to the matrix index
        for i in e:
            for j in e:
                if i != j:
                    DB[node_index[i], node_index[j]] += weight
    return DB, nodes

def compute_sir_threshold(DB, mu=0.05):
    """
    Compute critical infection rate for SIR: beta_crit = mu / lambda_max(DB)
    """
    # largest eigenvalue
    lambda_max = eigs(DB, k=1, which='LR', return_eigenvectors=False)[0].real
    # critical infection rate
    beta_crit = mu / lambda_max if lambda_max > 0 else np.inf
    return lambda_max, beta_crit

def summarize_hypergraph(H):
    """Print quick summary of structure"""
    degrees = [H.degree(node) for node in H.nodes]
    edge_sizes = [len(e) for e in H.edges.members()]
    z = np.mean(degrees)
    print(f"Mean hyperdegree (z): {z:.3f}")
    print(f"Average hyperedge size: {np.mean(edge_sizes):.3f}")
    print(f"Num nodes: {H.num_nodes}, Num edges: {H.num_edges}")

if __name__ == "__main__":
    # Load a sample hypergraph
    H = xgi.load_xgi_data("hospital-lyon", max_order=None)

    summarize_hypergraph(H)
    DB, nodes = compute_DB_matrix(H)

    # Assume recovery rate mu
    mu = 0.05
    lambda_max, beta_crit = compute_sir_threshold(DB, mu=mu)
    print(f"λ_max(DB): {lambda_max:.5f}")
    print(f"Critical infection rate beta_crit ≈ {beta_crit:.5f} for mu = {mu}")


Mean hyperdegree (z): 774.680
Average hyperedge size: 2.087
Num nodes: 75, Num edges: 27834
λ_max(DB): 1823.05156
Critical infection rate beta_crit ≈ 0.00003 for mu = 0.05


In [7]:
H = xgi.load_xgi_data("contact-primary-school", max_order=None)

summarize_hypergraph(H)
DB, nodes = compute_DB_matrix(H)

# Assume recovery rate mu
mu = 0.05
lambda_max, beta_crit = compute_sir_threshold(DB, mu=mu)
print(f"λ_max(DB): {lambda_max:.5f}")
print(f"Critical infection rate beta_crit ≈ {beta_crit:.5f} for mu = {mu}")

Mean hyperdegree (z): 925.612
Average hyperedge size: 2.096
Num nodes: 242, Num edges: 106879
λ_max(DB): 1412.99115
Critical infection rate beta_crit ≈ 0.00004 for mu = 0.05


In [12]:
for data in datasets:
    print(data)
    H = xgi.load_xgi_data(data, max_order=None)

    summarize_hypergraph(H)
    DB, nodes = compute_DB_matrix(H)

    # Assume recovery rate mu
    mu = 0.05
    lambda_max, beta_crit = compute_sir_threshold(DB, mu=mu)
    print(f"λ_max(DB): {lambda_max:.5f}")
    print(f"Critical infection rate beta_crit ≈ {beta_crit:.5f} for mu = {mu}\n")
    

contact-primary-school
Mean hyperdegree (z): 925.612
Average hyperedge size: 2.096
Num nodes: 242, Num edges: 106879
λ_max(DB): 1412.99115
Critical infection rate beta_crit ≈ 0.00004 for mu = 0.05

contact-high-school
Mean hyperdegree (z): 1078.648
Average hyperedge size: 2.050
Num nodes: 327, Num edges: 172035
λ_max(DB): 3041.02918
Critical infection rate beta_crit ≈ 0.00002 for mu = 0.05

hospital-lyon
Mean hyperdegree (z): 774.680
Average hyperedge size: 2.087
Num nodes: 75, Num edges: 27834
λ_max(DB): 1823.05156
Critical infection rate beta_crit ≈ 0.00003 for mu = 0.05

email-enron
Mean hyperdegree (z): 181.851
Average hyperedge size: 2.473
Num nodes: 148, Num edges: 10885
λ_max(DB): 779.25918
Critical infection rate beta_crit ≈ 0.00006 for mu = 0.05

email-eu
Mean hyperdegree (z): 559.800
Average hyperedge size: 2.391
Num nodes: 1005, Num edges: 235263
λ_max(DB): 4877.58277
Critical infection rate beta_crit ≈ 0.00001 for mu = 0.05

ndc-substances
Mean hyperdegree (z): 40.946
Avera